In [3]:
from Bio import SeqIO
from io import StringIO
from gzip import open as gzopen
from Bio.SeqIO.FastaIO import SimpleFastaParser
import numpy as np
#import matplotlib.pyplot as plt
import os 
import pandas as pd

pd.options.display.max_columns = None
#pd.options.display.max_rows = None

# Step:1 Extract AMPs and Non AMPs


In [18]:
amp_DB_fasta = "/home/kongkitimanonk/SCRATCH_NOBAK/datasets/phase2/amp_clustered100.22DBs.fasta"
uniref50 = "/home/kongkitimanonk/SCRATCH_NOBAK/datasets/phase2/uniref50.len2350.fasta"
non_AMPs_path='/home/kongkitimanonk/SCRATCH_NOBAK/datasets/phase2/non_AMPs.csv'
AMPs_path='/home/kongkitimanonk/SCRATCH_NOBAK/datasets/phase2/AMPs.csv'

## AMPs

In [5]:

lengths = map(len, SeqIO.parse(amp_DB_fasta, 'fasta'))
df = pd.Series([x for x in lengths], name="length").to_frame()

In [6]:
df.length.describe()

count    21226.000000
mean        60.496938
std        114.696874
min         11.000000
25%         18.000000
50%         28.000000
75%         61.000000
max       2337.000000
Name: length, dtype: float64

In [7]:
df['length'].value_counts(bins=100)

(8.673, 34.26]        12220
(34.26, 57.52]         3384
(57.52, 80.78]         2620
(80.78, 104.04]        1208
(127.3, 150.56]         218
                      ...  
(1592.68, 1615.94]        0
(1639.2, 1662.46]         0
(1662.46, 1685.72]        0
(1708.98, 1732.24]        0
(1150.74, 1174.0]         0
Name: length, Length: 100, dtype: int64

In [8]:
df_count=df.groupby('length').length.count().reset_index(name='count').sort_values(['count'], ascending=False)
#df_count.to_csv('df_count.csv', index=False)
_tmp_df_count =df_count
_tmp_df_count

,length,count
9,20,1131
2,13,1119
1,12,912
7,18,873
4,15,855
...,...,...
392,448,1
394,455,1
396,460,1
397,461,1


In [30]:
_tmp_df_count.loc[ _tmp_df_count['length'] == 20]['count'].values +1

9    1128
Name: count, dtype: int64

In [47]:
_tmp_df_count['count'].values.sum() == 0

False

write to csv file

In [17]:
AMPs_csv = open(AMPs_path, 'w')
AMPs_csv.write('"ID","Sequence","length"'+ '\n')
with open(amp_DB_fasta) as amps_record:
    for _name, _seq in  SimpleFastaParser(amps_record):
        AMPs_csv.write('"'+_name+'","'+_seq+'","' + str(len(_seq))+'"' + '\n')

## Non AMPs

In [ ]:
!seqkit --threads 20 -M 2350 /home/kongkitimanonk/SCRATCH_NOBAK/datasets/phase2/uniref50.fasta.gz > /home/kongkitimanonk/SCRATCH_NOBAK/datasets/phase2/uniref50.len2350.fasta

In [10]:
NonAMPs_csv = open(non_AMPs_path, 'w')
NonAMPs_csv.write('"ID","Sequence","length"'+ '\n')
with open(uniref50) as record_len2400:
      for _name, _seq in  SimpleFastaParser(record_len2400):
           seqLen = len(_seq)
           #update value
           _tmp_row = _tmp_df_count.loc[ _tmp_df_count['length'] == seqLen]
           if not _tmp_row.empty and  _tmp_row['count'].values > 0:
               #print (str(_tmp_row['length']) +" "+ str(_tmp_row['count']))
               NonAMPs_csv.write('"'+_name+'","'_seq+'","'+ str(seqLen)+'"' + '\n')
               _tmp_df_count.at[ _tmp_row.index,'count']= _tmp_row['count'] - 1
           else:
               if _tmp_df_count['count'].values.sum() == 0:
                   print("stop")
                   break

        


stop


In [ ]:
## Split file 

def batch_iterator(iterator, batch_size):
    """Returns lists of length batch_size.

    This can be used on any iterator, for example to batch up
    SeqRecord objects from Bio.SeqIO.parse(...), or to batch
    Alignment objects from Bio.AlignIO.parse(...), or simply
    lines from a file handle.

    This is a generator function, and it returns lists of the
    entries from the supplied iterator.  Each list will have
    batch_size entries, although the final list may be shorter.
    """
    entry = True  # Make sure we loop once
    while entry:
        batch = []
        while len(batch) < batch_size:
            try:
                entry =  next(iterator)
            except StopIteration:
                entry = None
            if entry is None:
                # End of file
                break
            batch.append(entry)
        if batch:
            yield batch
"""
record_iter = SeqIO.parse(gzopen("uniref50.fasta.gz", "rt"), "fasta")
for i, batch in enumerate(batch_iterator(record_iter, 200000)):
    filename = "group_%i.fasta" % (i + 1)
    with open(filename, "w") as handle:
        count = SeqIO.write(batch, handle, "fasta")
    print("Wrote %i records to %s" % (count, filename))
"""            

#  Step:2 Data labeling & Cleaning

In [ ]:
!pip install ipympl

In [53]:

import csv
import matplotlib.pyplot as plt

%matplotlib widget

In [31]:
#nonAMPs_df=pd.read_csv(non_AMPs_path,sep=',',header=0,quoting=csv.QUOTE_ALL)
nonAMPs_df=pd.read_csv(non_AMPs_path,sep=',',header=0)
AMPs_df=pd.read_csv(AMPs_path,sep=',',header=0,quoting=csv.QUOTE_ALL)

In [34]:
AMPs_df.head()

,ID,Sequence,length
0,EN92515250|C|B3FJD7|phage 201phi2-1 lysin,MAKKSVPLRKPAGSDGQGNIKVPGGPVVLDLGDFDDIFGPMESESP...,2337
1,EN54061055|C|F8SJ56|phage PhiPA3 lysin,MASKKTTLPKPKGINPQGSIVQLDLDDFDDLFDEDFGLPKKNSPYT...,2319
2,EN4815120|C|Q8SCY1|phage phiKZ lysin,MAKKVTLPKGQTGATGTTLGQAGNILDLSDVDDIFGDTPKAKKGSP...,2237
3,EN3016141|D|D2J8A7|bacteriocin,MAETIKGLRIDLSLKDMGVGRSITELKRSFRTLNSDLKVSSKNFEY...,1619
4,EN9175723|BD|Q93IM3|bacteriocin,MAKKKNTYKVPSIIALTLAGTALTTHHAQAADKTQDQSTNKNILND...,1564


In [33]:
nonAMPs_df.head()

,ID,Sequence,length
0,UniRef50_A0A672ZWI7 Ig-like domain-containing ...,MVIHQRHTSDESFSSSPVEIRITAATPIPELAEERSAEKPPAVTET...,189
1,UniRef50_A0A3Q2PZR6 Uncharacterized protein n=...,MSTQAPTFTQPLQSVVALEGSAATFEAQVSGSPVPEVSWFRDGQVL...,295
2,UniRef50_V9IMB4 Titin n=21 Tax=Apis TaxID=7459...,MWMEVGRVNADTQKLNIRDLQENHMYLIRIFAKNEVGLSDPLESDE...,121
3,UniRef50_B4IRR3 GL21409 (Fragment) n=18 Tax=Dr...,IIHPLEDVSVTKGENAIFELELSKGDALVKWFKNGKELVFNDRVQL...,320
4,UniRef50_UPI00083D1E21 titin-like n=26 Tax=Apo...,HDFGIVILDILYCYEENSGVYECRAFNKYGEDTTKATLKCFSKASL...,65


## Cleaning

In [ ]:
#

## Identity check of Distribution 

In [49]:
def lengthDistribution(df,title):
    fig = plt.figure()
    ax = fig.gca()
    pd.Series(df['length']).hist(color='blue', bins=200, ax=ax)
    plt.ylabel('Num of Seq with this length ')
    plt.xlabel('Length of Seq')
    plt.title(title)

In [52]:
lengthDistribution(nonAMPs_df,"Non Amps")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Labeling

In [ ]:
# setup
am